In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
url='https://www.cdslab.org/recipes/programming/stat-covmat/globalLandTempHist.txt'

In [3]:
#downloads file from given url
def download(url, fpath):
    import requests
    r = requests.get(url)
    with open(fpath, 'wb') as f:
        f.write(r.content)
    return None

#fetching data file
def fetchDatafromWeb(filePath="./globalLandTempHist.txt",
                         url= url):
    download(url,filePath)
    return filePath

def readFile(fpath):
    import pandas as pd
    return pd.read_csv(fpath, ", ") 


In [4]:
fetchDatafromWeb()

'./globalLandTempHist.txt'

In [5]:
df=readFile('./globalLandTempHist.txt')
df

<ipython-input-3-47e34b36e366>:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(fpath, ", ")


,year,anomaly
0,1743.846154,-1.131
1,1743.923077,NaN
2,1744.076923,NaN
3,1744.153846,NaN
4,1744.230769,NaN
...,...,...
3234,2013.384615,0.813
3235,2013.461538,1.088
3236,2013.538462,0.671
3237,2013.615385,0.919


In [6]:
#droping rows with values NaN
ndf=df.dropna()
ndf

,year,anomaly
0,1743.846154,-1.131
5,1744.307692,1.926
6,1744.384615,-1.115
7,1744.461538,-0.235
8,1744.538462,-0.430
...,...,...
3234,2013.384615,0.813
3235,2013.461538,1.088
3236,2013.538462,0.671
3237,2013.615385,0.919


In [10]:
data=ndf.to_numpy()

# 1. Prove that the diagonal elements of a correlation matrix of a dataset must be one

In [17]:
# calculates the covariance between X and y from given data
def cov(x, y):
    mean_x, mean_y = x.mean(), y.mean()
    n = len(x)
    
    return sum((x - mean_x) * (y - mean_y)) / (n-1)
 
    
# calculates the Covariance matrix for input data 
def genCovMat(data, Mean=None):
    
    """""
    Generate and return the covariance matrix of input data.
    The cols of the data must be individual values.
    The rows must be individual observations.
    Please pass clean matrix of all real values (0,NA, no NaN).
    
    Parameters
    ------------
        Data
            The input numpy matrix of data of all numeric values.
        Mean
            The mean of the input data along the columns (Attributes)
            (**optional**, default=np.mean(data))
        
    """""
 # get the rows and cols
    rows, cols = data.shape
    
 # the covariance matrix has a shape of n_attributes x n_attributes
    cov_mat = np.zeros((cols, cols))
    #nfact= 1/(len(data[:,0])-1)
    for i in range(cols):
        #for j in range (i+1):
        for j in range(cols):
                # store the value in the matrix
                cov_mat[i][j] = cov(data[:, i], data[:, j]) 
                #cov_mat[i][j] = nfact*np.dot(data[:,i]-mean[i],data[:,j]-mean[j])
    return cov_mat

# calculates Correlation matrix using Covariance function (cov()) for input data
def genCorMat(data):

    rows, cols = data.shape
    #corr_mat = np.ones((cols, cols))
    corr_mat = np.ones((cols, cols))
    
    for i in range(cols):
        #for j in range (i+1): 
        for j in range(cols): 
            #if i/=j: not equal

                x, y = data[:, i], data[:, j]
                # note: covariance matrix is just being normalized
                #corr_mat[i][j] = cov(x, y) / (x.std() * y.std())
                corr_mat[i,j] = data[i,j] / np.sqrt(data[i,i]*data[j,j])
                #corr_mat[j,i] = corr_mat[i,j] 

    return corr_mat

In [18]:
#using dropna
cov_matrix=genCovMat(data)
corr_matrix=genCorMat(cov_matrix)
print(cov_matrix)
print(corr_matrix)

[[5.81382831e+03 2.30956353e+01]
 [2.30956353e+01 8.75891017e-01]]
[[1.         0.32364863]
 [0.32364863 1.        ]]


# 2. Computing the Spearman's correlation coefficient of a dataset

In [7]:
def getRank(vector):
    lst= sorted(range(len(vector)), key=vector.__getitem__)
    return np.asarray(lst, dtype=np.float32)

In [12]:
import math
def getSpearmanCor(data1,data2):
    """""
        Generate and return the spearman correlation coefficient. 
        Input two columns of data to your function to compute the spearman corrcoeff. 
        The columns correspond to the data attributes.
        Make sure rows of data with "nan" are removed.
        
        Parameters:
            data1,data2 = data attribute 1, data attribute 2
    """""
    
    #get array ranks of X,y
    X_rank = getRank(data1)
    Y_rank = getRank(data2)
    
    xr_mean, yr_mean =  X_rank.mean(),  Y_rank.mean()
   

    xr_var = sum ((x_i - xr_mean)**2 for x_i in  X_rank)
    yr_var = sum ((y_i - yr_mean)**2 for y_i in  Y_rank)
    
    #cov(x,y)- numerator
    #sqrt(varx*vary)-denominator
    cov = sum((x_i-xr_mean)*(y_i-yr_mean) for x_i,y_i in zip( X_rank, Y_rank))
    den = math.sqrt(xr_var*yr_var)
    #den = x_std*y_std --doesnt work
        
    return cov/den

In [13]:
getSpearmanCor(data[:,0],data[:,1])

0.3848867684561194

In [16]:
#using pandas
ndf['year'].corr(ndf['anomaly'], method='spearman')

0.38458820147753553

In [15]:
import scipy.stats
##scipy.stats.spearmanr(x, y) returns (correlation rho, p-value)
scipy.stats.spearmanr(data[:,0],data[:,1]).correlation

0.38458820147753553

# 3.Which correlation coefficient is the most sensitive to outliers among the three Pearson, Spearman, and Kendall correlation coefficients? Why?

The most sensitive to outliers is Pearson's. This is because it measures a linear relation while the other two are more robust meassure of assoication (using ranking).

# 4. Computing the covariance matrix from the correlation matrix and standard deviations

In [24]:
print("Goal to go from corr_matrix to cov_matrix: \n")
print("Corr_matrix: \n",corr_matrix)
print("Cov_matrx: \n",cov_matrix)

Goal to go from corr_matrix to cov_matrix: 

Corr_matrix: 
 [[1.         0.32364863]
 [0.32364863 1.        ]]
Cov_matrx: 
 [[5.81382831e+03 2.30956353e+01]
 [2.30956353e+01 8.75891017e-01]]


In [74]:
std_vec=np.array([data[:,0].std(), data[:,1].std()])

In [71]:
def CovMatFromCorMat (CorMat, StdVec=None):
     """""
        Generate and return Covariance matrix. 
        Input correlation matrix and std vector.
        Make sure rows of data with "nan" are removed.
        
        Goal:
        cov_matrix= (std_diag matrix)* corr_matrix * (std_diag matrix)
    """""
    # get the rows and cols
    #rows, cols = CorMat.shape   
     # the covariance matrix has a shape of n_attributes x n_attributes
    #cov_mat = np.zeros((cols, cols))
     if StdVec is None:
        cov_mat=CorMat
     else:
        std_diag_matrix= np.diag(StdVec)
        mat_1=np.matmul(std_diag_matrix,CorMat)
        cov_mat=np.matmul(mat_1,std_diag_matrix)
    
     return cov_mat

In [72]:
CovMatFromCorMat (corr_matrix, StdVec=None)

array([[1.        , 0.32364863],
       [0.32364863, 1.        ]])

In [73]:
CovMatFromCorMat (corr_matrix, std_vec)

array([[5.81199256e+03, 2.30883427e+01],
       [2.30883427e+01, 8.75614448e-01]])